# Results

## Table of Contents
* [1)*American Put Prices* Applying Least-Square Monte Carlo with Longstaff-Schwartz Parameters](#first-bullet)
    - [1a) Black-Scholes (*Akin to Table 1 in Longstaff-Schwartz*)](#firsta-bullet)
    - [1b) Jump Merton](#firstb-bullet)
    - [1c) Constant Elasticity of Variance (CEV)](#firstc-bullet)
* [2) *American Put Prices* Applying Least-Square Monte Carlo with Our Own Parameter Values](#second-bullet)
    - [2a) Black-Scholes](#seconda-bullet)
    - [2b) Jump Merton](#secondb-bullet)
        - [2b extra) Jump Merton (BAD TRUNCATION)](#secondba-bullet)
    - [2c) Constant Elasticity of Variance (CEV)](#secondc-bullet)

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import os

# local
from DC import DynamicChebyshev
from LSM import MonteCarloOptionPricing
from FD import AmericanPutFiniteDifference

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1) *American Put Prices* Applying Least-Square Monte Carlo with Longstaff-Schwartz Parameters <a class="anchor" id="first-bullet"></a>
The simulation is based on 100,000 (50,000 plus 50,000 antithetic) paths for the stock price.

In [5]:
# Parameter inputs
r = 0.06  # risk-free rate
K = 40.   # strike price
λ = 0.4 # intensity rate of the Poisson process
dim = 10_000  # number of simulation paths
n = 50  # number of exercise rights per year
seed = 15_001  # random seed for reproducibility
n_chebyshev_pol = 100
use_AV = True  # antithetic variates
poly_degree = 3  # polynomial degree for LSM regression

In [6]:
# Varying option parameters
S0_values = [36, 38, 40, 42, 44]
σ_values = [0.20, 0.40]
T_values = [1, 2]

### 1a) Black-Scholes (*Akin to Table 1 in Longstaff-Schwartz*) <a class="anchor" id="firsta-bullet"></a>


In [7]:
# Store results
Option_Pricing_Results_BS = []

# Iterate over each combination of S, sigma, and T
for S0 in S0_values:
    for σ in σ_values:
        for T in T_values:
            # Create instance from class
            MC_Option_Pricing_BS = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)
            
            # 1) Calculate European Put using Black-Scholes
            BS_Price = MC_Option_Pricing_BS.BS_option_value('put')
            
            # 2) Compute American Put using LSM
            MC_Option_Pricing_BS.GeometricBrownianMotion_vec() # simulate GBM paths
            American_put_price = MC_Option_Pricing_BS.american_option_LSM(poly_degree, otype='put') # price option
            
            # 3) Calculate premium
            Option_Premium = American_put_price[0] - BS_Price 
            
            # 4) Compute American Put using Implicit Finite Difference
            Implicit_Finite_Difference = AmericanPutFiniteDifference(K, r, M = 1_000)
            FD = Implicit_Finite_Difference.implicit_FD(S0, σ, T, N=n)
            
            # 5) Compute American Put using Dynamic Chebyshev 
            DC_option_pricing = DynamicChebyshev(r, S0, K, T, σ, λ, dim, n, n_chebyshev_pol, seed, use_AV)
            
            # a) offline phase
            domain = DC_option_pricing.calculate_truncated_domain_GBM()
            xknots = DC_option_pricing.calculate_nodal_points(domain)
            BS = DC_option_pricing.generate_GBM_path(xknots)
            Γ = DC_option_pricing.compute_generalized_moments(domain, xknots)

            # b) online phase
            DC = DC_option_pricing.price_option_with_dynamic_chebyshev(xknots, Γ)
            
            # 6) Store the results
            Option_Pricing_Results_BS.append((S0, σ, T, BS_Price, American_put_price[0], Option_Premium, FD, American_put_price[3], DC[0], DC[1]))
            
            # Print
            print(f'S0/Sigma/T: {S0}/{σ}/{T}, EUR Put: {BS_Price:.4f}, LSM: {American_put_price[0]:.4f}, Premium: {Option_Premium:.4f}, F.D.: {FD:.4f}, s.e.: {American_put_price[3]:.4f}, D.C.: {DC[0]:.4f}, s.e.: {DC[1]:.4f}')

S0/Sigma/T: 36/0.2/1, EUR Put: 3.8443, LSM: 4.4760, Premium: 0.6317, F.D.: 4.4650, s.e.: 0.0291, D.C.: 4.5445, s.e.: 0.0772
S0/Sigma/T: 36/0.2/2, EUR Put: 3.7630, LSM: 4.8401, Premium: 1.0771, F.D.: 4.8114, s.e.: 0.0358, D.C.: 4.9248, s.e.: 0.0934
S0/Sigma/T: 36/0.4/1, EUR Put: 6.7114, LSM: 7.0991, Premium: 0.3877, F.D.: 7.0754, s.e.: 0.0614, D.C.: 7.2365, s.e.: 0.1195
S0/Sigma/T: 36/0.4/2, EUR Put: 7.7000, LSM: 8.4803, Premium: 0.7802, F.D.: 8.3893, s.e.: 0.0728, D.C.: 8.6651, s.e.: 0.1397
S0/Sigma/T: 38/0.2/1, EUR Put: 2.8519, LSM: 3.2598, Premium: 0.4079, F.D.: 3.2368, s.e.: 0.0309, D.C.: 3.3359, s.e.: 0.0723
S0/Sigma/T: 38/0.2/2, EUR Put: 2.9906, LSM: 3.7552, Premium: 0.7646, F.D.: 3.7185, s.e.: 0.0361, D.C.: 3.8472, s.e.: 0.0891
S0/Sigma/T: 38/0.4/1, EUR Put: 5.8343, LSM: 6.1525, Premium: 0.3182, F.D.: 6.1224, s.e.: 0.0601, D.C.: 6.2928, s.e.: 0.1168
S0/Sigma/T: 38/0.4/2, EUR Put: 6.9788, LSM: 7.6809, Premium: 0.7021, F.D.: 7.5687, s.e.: 0.0723, D.C.: 7.8383, s.e.: 0.1380
S0/Sigma

### 1b) Jump Merton <a class="anchor" id="firstb-bullet"></a>

In [8]:
λ = 0.4 # intensity rate of the Poisson process
α = -0.5 # Mean of log-normal jump size
β = 0.4 # Volatility of log-normal jump size
σ = 0.2 # diffusion parameter

In [9]:
# Store results
Option_Pricing_Results_JM = []

# Iterate over each combination of S and T
for S0 in S0_values:
    for T in T_values:
        # Create instance from class
        MC_Option_Pricing_JM = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)

        # 1) Calculate European Put using Jump Merton semi-closed solution
        JumpMerton_price = MC_Option_Pricing_JM.merton_jump_option_value(α, β)

        # 2) Compute American Put using LSM
        MC_Option_Pricing_JM.MertonJumpDiffusion_vec(α, β) # simulate jump paths
        American_put_price = MC_Option_Pricing_JM.american_option_LSM(poly_degree, otype='put') # price option

        # 3) Calculate premium
        Option_Premium = American_put_price[0] - JumpMerton_price 

        # 4) Compute American Put using Dynamic Chebyshev 
        DC_option_pricing = DynamicChebyshev(r, S0, K, T, σ, λ, dim, n, n_chebyshev_pol, seed, use_AV)
        # a) offline phase
        domain = DC_option_pricing.calculate_truncated_domain_JumpMerton(α, β)
        xknots = DC_option_pricing.calculate_nodal_points(domain)
        JM = DC_option_pricing.generate_Jump_path(xknots, α, β)
        Γ = DC_option_pricing.compute_generalized_moments(domain,xknots)

        # b) online phase
        DC = DC_option_pricing.price_option_with_dynamic_chebyshev(xknots,Γ)

        # 5) Store the results
        Option_Pricing_Results_JM.append((S0, σ, T, JumpMerton_price, American_put_price[0], Option_Premium, DC[0]))

        # Print
        print(f'S0/Sigma/T: {S0}/{T}, EUR Put: {JumpMerton_price:.4f}, LSM: {American_put_price[0]:.4f}, D.C.: {DC[0]:.4f}')

S0/Sigma/T: 36/1, EUR Put: 5.7636, LSM: 6.2070, D.C.: 5.8151
S0/Sigma/T: 36/2, EUR Put: 7.1273, LSM: 8.1578, D.C.: 8.0533
S0/Sigma/T: 38/1, EUR Put: 5.0684, LSM: 5.4040, D.C.: 4.9568
S0/Sigma/T: 38/2, EUR Put: 6.5609, LSM: 7.5066, D.C.: 7.3789
S0/Sigma/T: 40/1, EUR Put: 4.5068, LSM: 4.7774, D.C.: 4.2746
S0/Sigma/T: 40/2, EUR Put: 6.0619, LSM: 6.9193, D.C.: 6.7883
S0/Sigma/T: 42/1, EUR Put: 4.0494, LSM: 4.2979, D.C.: 3.7218
S0/Sigma/T: 42/2, EUR Put: 5.6189, LSM: 6.4151, D.C.: 6.2650
S0/Sigma/T: 44/1, EUR Put: 3.6710, LSM: 3.8963, D.C.: 3.2744
S0/Sigma/T: 44/2, EUR Put: 5.2229, LSM: 5.9829, D.C.: 5.7965


### 1c) Constant Elasticity of Variance (CEV) <a class="anchor" id="firstc-bullet"></a>

In [10]:
γ = 1.5 # parameter governing elasticity with respect to price, if set = 2 then we obtain BS prices

In [11]:
# Store results
Option_Pricing_Results_CEV = []

# Iterate over each combination of S and T
for S0 in S0_values:
    for T in T_values:
        # Create instance from class
        MC_Option_Pricing_CEV = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)

        # 1) Compute American Put using LSM
        MC_Option_Pricing_CEV.CEV(γ) # simulate CEV paths
        American_put_price = MC_Option_Pricing_CEV.american_option_LSM(poly_degree, otype='put') # price option

        # 2) Compute American Put using Dynamic Chebyshev
        DC_option_pricing = DynamicChebyshev(r, S0, K, T, σ, λ, dim, n, n_chebyshev_pol, seed, use_AV)
        # a) offline phase
        domain = DC_option_pricing.calculate_truncated_domain_GBM()
        xknots = DC_option_pricing.calculate_nodal_points(domain)
        CEV = DC_option_pricing.generate_CEV_path(xknots, γ)
        Γ = DC_option_pricing.compute_generalized_moments(domain, xknots)

        # b) online phase
        DC = DC_option_pricing.price_option_with_dynamic_chebyshev(xknots, Γ)

        # 3) Store the results
        Option_Pricing_Results_CEV.append((S0, σ, T, American_put_price[0], DC[0]))

        # Print
        print(f'S0/Sigma/T: {S0}/{T}, LSM: {American_put_price[0]:.4f}, D.C.: {DC[0]:.4f}')

S0/Sigma/T: 36/1, LSM: 3.9521, D.C.: 4.0606
S0/Sigma/T: 36/2, LSM: 3.9042, D.C.: 4.1675
S0/Sigma/T: 38/1, LSM: 1.9563, D.C.: 2.6215
S0/Sigma/T: 38/2, LSM: 1.9376, D.C.: 2.8772
S0/Sigma/T: 40/1, LSM: 0.6246, D.C.: 1.6299
S0/Sigma/T: 40/2, LSM: 0.6965, D.C.: 1.9655
S0/Sigma/T: 42/1, LSM: 0.1642, D.C.: 0.9772
S0/Sigma/T: 42/2, LSM: 0.2336, D.C.: 1.3288
S0/Sigma/T: 44/1, LSM: 0.0359, D.C.: 0.5641
S0/Sigma/T: 44/2, LSM: 0.0729, D.C.: 0.8894


## 2) *American Put Prices* Applying Least-Square Monte Carlo with Our Own Parameter Values <a class="anchor" id="second-bullet"></a>
The simulation is based on 100,000 (50,000 plus 50,000 antithetic) paths for the stock price.

In [12]:
# Parameter inputs
r = 0.06  # risk-free rate
K = 100.   # strike price
λ = 0.4 # intensity rate of the Poisson process
dim = 10_000  # number of simulation paths
n = 252  # number of exercise rights per year
n_chebyshev_pol = 100
seed = 15_001  # random seed for reproducibility
use_AV = True  # antithetic variates
poly_degree = 3  # polynomial degree for LSM regression

In [13]:
# Option parameters
S0_values = [90, 95, 100, 105, 110]
σ_values = [0.20, 0.40]
T_values = [1, 2]

## 2a) Black-Scholes <a class="anchor" id="seconda-bullet"></a>

In [14]:
# Store results
Results_BS = []

# Store results
Option_Pricing_Results_BS = []

# Iterate over each combination of S, sigma, and T
for S0 in S0_values:
    for σ in σ_values:
        for T in T_values:
            # Create instance from class
            MC_Option_Pricing_BS = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)
            
            # 1) Calculate European Put using Black-Scholes
            BS_Price = MC_Option_Pricing_BS.BS_option_value('put')
            
            # 2) Compute American Put using LSM
            MC_Option_Pricing_BS.GeometricBrownianMotion_vec() # simulate GBM paths
            American_put_price = MC_Option_Pricing_BS.american_option_LSM(poly_degree, otype='put') # price option
            
            # 3) Calculate premium
            Option_Premium = American_put_price[0] - BS_Price 
            
            # 4) Compute American Put using Implicit Finite Difference
            Implicit_Finite_Difference = AmericanPutFiniteDifference(K, r, M = 1_000)
            FD = Implicit_Finite_Difference.implicit_FD(S0, σ, T, N=n)
            
            # 5) Compute American Put using Dynamic Chebyshev 
            DC_option_pricing = DynamicChebyshev(r, S0, K, T, σ, λ, dim, n, n_chebyshev_pol, seed, use_AV)
            
            # a) offline phase
            domain = DC_option_pricing.calculate_truncated_domain_GBM()
            xknots = DC_option_pricing.calculate_nodal_points(domain)
            BS = DC_option_pricing.generate_GBM_path(xknots)
            Γ = DC_option_pricing.compute_generalized_moments(domain, xknots)

            # b) online phase
            DC = DC_option_pricing.price_option_with_dynamic_chebyshev(xknots, Γ)
            
            # 6) Store the results
            Results_BS.append((S0, σ, T, BS_Price, American_put_price[0], Option_Premium, FD, American_put_price[3], DC[0], DC[1]))
            
            # Print
            print(f'S0/Sigma/T: {S0}/{σ}/{T}, EUR Put: {BS_Price:.4f}, LSM: {American_put_price[0]:.4f}, Premium: {Option_Premium:.4f}, F.D.: {FD:.4f}, s.e.: {American_put_price[3]:.4f}, D.C.: {DC[0]:.4f}, s.e.: {DC[1]:.4f}')

S0/Sigma/T: 90/0.2/1, EUR Put: 9.6108, LSM: 11.2581, Premium: 1.6474, F.D.: 11.2053, s.e.: 0.0750, D.C.: 12.0103, s.e.: 0.1923
S0/Sigma/T: 90/0.2/2, EUR Put: 9.4075, LSM: 12.1344, Premium: 2.7269, F.D.: 12.1015, s.e.: 0.0886, D.C.: 13.2345, s.e.: 0.2326
S0/Sigma/T: 90/0.4/1, EUR Put: 16.7785, LSM: 17.8591, Premium: 1.0806, F.D.: 17.7493, s.e.: 0.1446, D.C.: 19.2846, s.e.: 0.2974
S0/Sigma/T: 90/0.4/2, EUR Put: 19.2501, LSM: 21.4514, Premium: 2.2013, F.D.: 21.0758, s.e.: 0.1750, D.C.: 23.1401, s.e.: 0.3477
S0/Sigma/T: 95/0.2/1, EUR Put: 7.1298, LSM: 8.2376, Premium: 1.1078, F.D.: 8.1321, s.e.: 0.0734, D.C.: 9.1349, s.e.: 0.1800
S0/Sigma/T: 95/0.2/2, EUR Put: 7.4764, LSM: 9.4766, Premium: 2.0002, F.D.: 9.3612, s.e.: 0.0887, D.C.: 10.6659, s.e.: 0.2220
S0/Sigma/T: 95/0.4/1, EUR Put: 14.5858, LSM: 15.5552, Premium: 0.9694, F.D.: 15.3660, s.e.: 0.1420, D.C.: 17.0226, s.e.: 0.2906
S0/Sigma/T: 95/0.4/2, EUR Put: 17.4470, LSM: 19.3706, Premium: 1.9236, F.D.: 19.0214, s.e.: 0.1728, D.C.: 21.1619

## 2b) Jump Merton <a class="anchor" id="secondb-bullet"></a>

In [15]:
λ = 0.4 # intensity rate of the Poisson process
α = -0.5 # Mean of log-normal jump size
β = 0.4 # Volatility of log-normal jump size
σ = 0.2 # diffusion parameter

In [16]:
# Store results
Results_JM = []

# Iterate over each combination of S and T
for S0 in S0_values:
    for T in T_values:
        # Create instance from class
        MC_Option_Pricing_JM = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)

        # 1) Calculate European Put using Jump Merton semi-closed solution
        JumpMerton_price = MC_Option_Pricing_JM.merton_jump_option_value(α, β)

        # 2) Compute American Put using LSM
        MC_Option_Pricing_JM.MertonJumpDiffusion_vec(α, β) # simulate jump paths
        American_put_price = MC_Option_Pricing_JM.american_option_LSM(poly_degree, otype='put') # price option

        # 3) Calculate premium
        Option_Premium = American_put_price[0] - JumpMerton_price 

        # 4) Compute American Put using Dynamic Chebyshev 
        DC_option_pricing = DynamicChebyshev(r, S0, K, T, σ, λ, dim, n, n_chebyshev_pol, seed, use_AV)
        # a) offline phase
        domain = DC_option_pricing.calculate_truncated_domain_JumpMerton(α, β)
        xknots = DC_option_pricing.calculate_nodal_points(domain)
        JM = DC_option_pricing.generate_Jump_path(xknots, α, β)
        Γ = DC_option_pricing.compute_generalized_moments(domain,xknots)

        # b) online phase
        DC = DC_option_pricing.price_option_with_dynamic_chebyshev(xknots,Γ)

        # 5) Store the results
        Results_JM.append((S0, T, JumpMerton_price, American_put_price[0], Option_Premium, DC[0]))

        # Print
        print(f'S0/Sigma/T: {S0}/{T}, EUR Put: {JumpMerton_price:.4f}, LSM: {American_put_price[0]:.4f}, D.C.: {DC[0]:.4f}')

S0/Sigma/T: 90/1, EUR Put: 14.4089, LSM: 15.5227, D.C.: 16.9176
S0/Sigma/T: 90/2, EUR Put: 17.8182, LSM: 20.3477, D.C.: 22.7808
S0/Sigma/T: 95/1, EUR Put: 12.6710, LSM: 13.5883, D.C.: 14.7929
S0/Sigma/T: 95/2, EUR Put: 16.4024, LSM: 18.6529, D.C.: 21.0232
S0/Sigma/T: 100/1, EUR Put: 11.2670, LSM: 12.0897, D.C.: 12.9719
S0/Sigma/T: 100/2, EUR Put: 15.1547, LSM: 17.1334, D.C.: 19.4376
S0/Sigma/T: 105/1, EUR Put: 10.1234, LSM: 10.9404, D.C.: 11.3995
S0/Sigma/T: 105/2, EUR Put: 14.0472, LSM: 15.8141, D.C.: 18.0007
S0/Sigma/T: 110/1, EUR Put: 9.1776, LSM: 9.9055, D.C.: 10.0525
S0/Sigma/T: 110/2, EUR Put: 13.0571, LSM: 14.6897, D.C.: 16.6917


### 2b extra) Jump Merton (BAD TRUNCATION) <a class="anchor" id="secondba-bullet"></a>

In [24]:
# Store results
Results_JM = []

# Iterate over each combination of S and T
for S0 in S0_values:
    for T in T_values:
        # Create instance from class
        MC_Option_Pricing_JM = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)

        # 1) Calculate European Put using Jump Merton semi-closed solution
        JumpMerton_price = MC_Option_Pricing_JM.merton_jump_option_value(α, β)

        # 2) Compute American Put using LSM
        MC_Option_Pricing_JM.MertonJumpDiffusion_vec(α, β) # simulate jump paths
        American_put_price = MC_Option_Pricing_JM.american_option_LSM(poly_degree, otype='put') # price option

        # 3) Calculate premium
        Option_Premium = American_put_price[0] - JumpMerton_price 

        # 4) Compute American Put using Dynamic Chebyshev 
        DC_option_pricing = DynamicChebyshev(r, S0, K, T, σ, λ, dim, n, n_chebyshev_pol, seed, use_AV)
        # a) offline phase
        domain = DC_option_pricing.calculate_truncated_domain_GBM()
        xknots = DC_option_pricing.calculate_nodal_points(domain)
        JM = DC_option_pricing.generate_Jump_path(xknots, α, β)
        Γ = DC_option_pricing.compute_generalized_moments(domain,xknots)

        # b) online phase
        DC = DC_option_pricing.price_option_with_dynamic_chebyshev(xknots,Γ)

        # 5) Store the results
        Results_JM.append((S0, T, JumpMerton_price, American_put_price[0], Option_Premium, DC[0]))

        # Print
        print(f'S0/Sigma/T: {S0}/{T}, EUR Put: {JumpMerton_price:.4f}, LSM: {American_put_price[0]:.4f}, D.C.: {DC[0]:.4f}')

S0/Sigma/T: 90/1, EUR Put: 14.4089, LSM: 15.5227, D.C.: 12.3997
S0/Sigma/T: 90/2, EUR Put: 17.8182, LSM: 20.3477, D.C.: 16.7458
S0/Sigma/T: 95/1, EUR Put: 12.6710, LSM: 13.5883, D.C.: 9.8403
S0/Sigma/T: 95/2, EUR Put: 16.4024, LSM: 18.6529, D.C.: 14.7104
S0/Sigma/T: 100/1, EUR Put: 11.2670, LSM: 12.0897, D.C.: 7.8315
S0/Sigma/T: 100/2, EUR Put: 15.1547, LSM: 17.1334, D.C.: 12.9614
S0/Sigma/T: 105/1, EUR Put: 10.1234, LSM: 10.9404, D.C.: 6.2480
S0/Sigma/T: 105/2, EUR Put: 14.0472, LSM: 15.8141, D.C.: 11.4383
S0/Sigma/T: 110/1, EUR Put: 9.1776, LSM: 9.9055, D.C.: 4.9901
S0/Sigma/T: 110/2, EUR Put: 13.0571, LSM: 14.6897, D.C.: 10.0982


## 2c) Constant Elasticity of Variance (CEV) <a class="anchor" id="secondc-bullet"></a>

In [17]:
γ = 1.5 # parameter governing elasticity with respect to price, if set = 2 then we obtain BS prices

In [18]:
# Store results
Results_CEV = []

# Iterate over each combination of S and T
for S0 in S0_values:
    for T in T_values:
        # Create instance from class
        MC_Option_Pricing_CEV = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)

        # 1) Compute American Put using LSM
        MC_Option_Pricing_CEV.CEV(γ) # simulate CEV paths
        American_put_price = MC_Option_Pricing_CEV.american_option_LSM(poly_degree, otype='put') # price option

        # 2) Compute American Put using Dynamic Chebyshev
        DC_option_pricing = DynamicChebyshev(r, S0, K, T, σ, λ, dim, n, n_chebyshev_pol, seed, use_AV)
        # a) offline phase
        domain = DC_option_pricing.calculate_truncated_domain_GBM()
        xknots = DC_option_pricing.calculate_nodal_points(domain)
        CEV = DC_option_pricing.generate_CEV_path(xknots, γ)
        Γ = DC_option_pricing.compute_generalized_moments(domain, xknots)

        # b) online phase
        DC = DC_option_pricing.price_option_with_dynamic_chebyshev(xknots, Γ)

        # 3) Store the results
        Results_CEV.append((S0, σ, T, American_put_price[0], DC[0]))

        # Print
        print(f'S0/Sigma/T: {S0}/{T}, LSM: {American_put_price[0]:.4f}, D.C.: {DC[0]:.4f}')

S0/Sigma/T: 90/1, LSM: 9.9762, D.C.: 10.6231
S0/Sigma/T: 90/2, LSM: 9.9524, D.C.: 11.1944
S0/Sigma/T: 95/1, LSM: 4.9761, D.C.: 7.3549
S0/Sigma/T: 95/2, LSM: 4.9524, D.C.: 8.2602
S0/Sigma/T: 100/1, LSM: 1.0856, D.C.: 4.9656
S0/Sigma/T: 100/2, LSM: 1.1671, D.C.: 6.0541
S0/Sigma/T: 105/1, LSM: 0.1635, D.C.: 3.2703
S0/Sigma/T: 105/2, LSM: 0.2217, D.C.: 4.4112
S0/Sigma/T: 110/1, LSM: 0.0201, D.C.: 2.0961
S0/Sigma/T: 110/2, LSM: 0.0411, D.C.: 3.1921
